# LangChainBitcoin: LLM Agent Bitcoin Tools

This notebook shows some examples of how to use the set of "tools" provided in the `LangChainBitcoin` project with the `langchain` project and also the newly announced native function calls provided by the OpenAPI GPt-3/4 APIs. By hooking up Lightning tools directly to the LLM agent, we give it the ability to: interact with the Bitcoin network, hold/send/recv Bitcoin (on-chain or over LN), and also eventually directly mmanage/drive an active node.

Earlier versions of the `langchain` project included support for "tools" which an agent can use to figure out how to answer a prompt or iterate towards a goal. This was implemented via a series of prompts and chained LLMs to "teach" the LLM how to use a set of tools. 

On June 13th, 2023 [OpenAPI released support for native function calls](https://openai.com/blog/function-calling-and-other-api-updates)! Native function calls can be used to directly give the LLM native tools that it can use. Basic usage of thes function calls can be used to ensure that the agent emits structred data like JSON, which can be difficult produce reliably otherwise. The latest releases of the `langchain` library augment the prior abstractions with direct support for the new function API provided by Open AI. 

By combining the new function call API with `langchain`, and also our custom set of Bitcoin/Lightning tools, we're able to create agents that are able to directly interact with Bitcoin and also the Lightning Network. At a high level, we create some functions implementing tools, then _describe_ these tools to the agent. A description can include a strict schema for the input arguments (`a: string, b: Invoice`, etc) and also some few-shot examples of what a response would look like. 

In the rest of this notebook, we'll demo some basic tools to show how things are initialized, and also how to create and prompt the agent. 

We'll start with some imports needed to get the ball rolling:

In [2]:
from bitcoin_tools import LndTools
from lightning import LndNode
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI

Next we'll create a connection to our lnd node. This uses some dummy data, but be sure slot in proper macaroon, TLS, and connection information. In the future, this'll use LNC to make it much easier to pair to a node:

In [3]:
lnd_node = LndNode(
    cert_path='~/gocode/src/github.com/lightningnetwork/lnd/test_lnd2/tls.cert',
    macaroon_path='~/gocode/src/github.com/lightningnetwork/lnd/test_lnd2/data/chain/bitcoin/simnet/admin.macaroon',
    host='localhost',
    port=10018
)

Next we'll initialize our set of Bitcoin tools, passing in the live `lnd_node` connection above:

In [4]:
lnd_tools = LndTools.from_lnd_node(lnd_node=lnd_node)

The final part of our set up will create an instance of the GPT 3.5 model with a cut off date of June 13th. This date is important as only this version of the model knows how to make function calls. GPT 4 works too, but it's a bit slower, and we're not doing anything too fancy here. 

In [5]:
llm_model = ChatOpenAI(model="gpt-3.5-turbo-0613")

bitcoin_agent = initialize_agent(
    lnd_tools.get_tools(),
    llm_model,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

With our agent initialized, let's try out some example prompts!

First, let's see if it can figure out some basic information abuot the node that it's connected to:

In [6]:
print(bitcoin_agent.run('What is my current channel balance in satoshis?'))



> Entering new  chain...

Invoking: `channel_balance` with `{}`


balance: 30399
local_balance {
  sat: 30399
  msat: 30399000
}
remote_balance {
  sat: 966131
  msat: 966131000
}
unsettled_local_balance {
}
unsettled_remote_balance {
}
pending_open_local_balance {
}
pending_open_remote_balance {
}
Your current channel balance is 30,399 satoshis.

> Finished chain.
Your current channel balance is 30,399 satoshis.


In [7]:
print(bitcoin_agent.run('What is my current on chain balance'))



> Entering new  chain...

Invoking: `wallet_balance` with `{}`


total_balance: 1000
confirmed_balance: 1000
reserved_balance_anchor_chan: 10000
account_balance {
  key: "imported"
  value {
    confirmed_balance: 1000
  }
}
account_balance {
  key: "default"
  value {
  }
}
Your current on-chain balance is 1000 satoshis.

> Finished chain.
Your current on-chain balance is 1000 satoshis.


In [8]:
print(bitcoin_agent.run('Do I have any unconfirmed change in the node?'))



> Entering new  chain...

Invoking: `wallet_balance` with `{}`


total_balance: 1000
confirmed_balance: 1000
reserved_balance_anchor_chan: 10000
account_balance {
  key: "imported"
  value {
    confirmed_balance: 1000
  }
}
account_balance {
  key: "default"
  value {
  }
}
No, there is no unconfirmed change in the node. The total balance is 1000 satoshis, and the confirmed balance is also 1000 satoshis.

> Finished chain.
No, there is no unconfirmed change in the node. The total balance is 1000 satoshis, and the confirmed balance is also 1000 satoshis.


In [9]:
print(bitcoin_agent.run('what version of lnd are you hooked up to?'))



> Entering new  chain...

Invoking: `get_info` with `{}`


version: "0.16.99-beta commit=v0.16.0-beta-350-gbbbf7d33f"
commit_hash: "bbbf7d33fb1527acebb44e2a69d16fbcf24cc2fa"
identity_pubkey: "0231c1b36b280d1ecccff2fd7526b8188b83f918eb549b182f6d1d0c577bfede4c"
alias: "0231c1b36b280d1ecccf"
color: "#3399ff"
num_active_channels: 1
num_peers: 1
block_height: 3305
block_hash: "1931aa2f19b59676edfef04dbbf82ddcf878297f0db27490011f2f790837c372"
best_header_timestamp: 1686961637
synced_to_graph: true
chains {
  chain: "bitcoin"
  network: "simnet"
}
features {
  key: 0
  value {
    name: "data-loss-protect"
    is_required: true
    is_known: true
  }
}
features {
  key: 5
  value {
    name: "upfront-shutdown-script"
    is_known: true
  }
}
features {
  key: 7
  value {
    name: "gossip-queries"
    is_known: true
  }
}
features {
  key: 9
  value {
    name: "tlv-onion"
    is_known: true
  }
}
features {
  key: 12
  value {
    name: "static-remote-key"
    is_required: true
    is_know

So it can figure out which function it needs to call in order to figure how to execute/satsify the prompt. Let's trying some stuff that's a bit more LN specific:

In [10]:
invoice = "lnsb100u1pjfz83lpp55h08akqscecxejh5jfecdx7hxf693tsv7c3lyp207fjqwmx9a8fsdqqcqzzsxqyz5vqsp5xhdak7n7h2dtl0gky9xzffnwel67h94jawkn0skpkj4xmcvtyphq9qyyssqrk86elwl2wctg6pw4uxjh4a3swrpljfvefy3xcxxk90tu484wxrpyu5pcwdfk65yjgw4fash3dsyuyxdyalengm4pwwj8t083cgqyhcphvaj27"
prompt = "How many satoshis is this invoice requesting: {invoice}".format(invoice=invoice)
print(bitcoin_agent.run(prompt))



> Entering new  chain...

Invoking: `decode_invoice` with `{'invoice': 'lnsb100u1pjfz83lpp55h08akqscecxejh5jfecdx7hxf693tsv7c3lyp207fjqwmx9a8fsdqqcqzzsxqyz5vqsp5xhdak7n7h2dtl0gky9xzffnwel67h94jawkn0skpkj4xmcvtyphq9qyyssqrk86elwl2wctg6pw4uxjh4a3swrpljfvefy3xcxxk90tu484wxrpyu5pcwdfk65yjgw4fash3dsyuyxdyalengm4pwwj8t083cgqyhcphvaj27'}`


destination: "038cf9360fa60a6de14ad7490950f90e06e828d4557a711aa54fabc6a5f2b4d81d"
payment_hash: "a5de7ed810c6706ccaf49273869bd7327458ae0cf623f2054ff264076cc5e9d3"
num_satoshis: 10000
timestamp: 1687232063
expiry: 86400
cltv_expiry: 80
payment_addr: "5\333\333z~\272\232\277\275\026!L$\246n\317\365\353\226\262\353\2557\302\301\264\252m\341\213 n"
num_msat: 10000000
features {
  key: 9
  value {
    name: "tlv-onion"
    is_known: true
  }
}
features {
  key: 14
  value {
    name: "payment-addr"
    is_required: true
    is_known: true
  }
}
features {
  key: 17
  value {
    name: "multi-path-payments"
    is_known: true
  }
}
The invoice is requesting 10

In [11]:
prompt = "What else can you tell me about this Lightning invoice? Your response should be in natural language, as if you were describing the payment requirements to a customer. {invoice}".format(invoice=invoice)

print(bitcoin_agent.run(prompt))



> Entering new  chain...

Invoking: `decode_invoice` with `{'invoice': 'lnsb100u1pjfz83lpp55h08akqscecxejh5jfecdx7hxf693tsv7c3lyp207fjqwmx9a8fsdqqcqzzsxqyz5vqsp5xhdak7n7h2dtl0gky9xzffnwel67h94jawkn0skpkj4xmcvtyphq9qyyssqrk86elwl2wctg6pw4uxjh4a3swrpljfvefy3xcxxk90tu484wxrpyu5pcwdfk65yjgw4fash3dsyuyxdyalengm4pwwj8t083cgqyhcphvaj27'}`


destination: "038cf9360fa60a6de14ad7490950f90e06e828d4557a711aa54fabc6a5f2b4d81d"
payment_hash: "a5de7ed810c6706ccaf49273869bd7327458ae0cf623f2054ff264076cc5e9d3"
num_satoshis: 10000
timestamp: 1687232063
expiry: 86400
cltv_expiry: 80
payment_addr: "5\333\333z~\272\232\277\275\026!L$\246n\317\365\353\226\262\353\2557\302\301\264\252m\341\213 n"
num_msat: 10000000
features {
  key: 9
  value {
    name: "tlv-onion"
    is_known: true
  }
}
features {
  key: 14
  value {
    name: "payment-addr"
    is_required: true
    is_known: true
  }
}
features {
  key: 17
  value {
    name: "multi-path-payments"
    is_known: true
  }
}


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 976ee90ffe8a550092ca828c03f191f6 in your message.).


This Lightning invoice is for a payment of 10,000 satoshis (0.0001 BTC). The payment is to be made to the destination address "038cf9360fa60a6de14ad7490950f90e06e828d4557a711aa54fabc6a5f2b4d81d". The payment hash is "a5de7ed810c6706ccaf49273869bd7327458ae0cf623f2054ff264076cc5e9d3".

The invoice has a timestamp of 1687232063, indicating when it was created. It will expire in 86400 seconds (24 hours). The CLTV expiry is set to 80, which means the payment must be completed before a certain block height.

The payment address is a series of characters that uniquely identifies the payment. It is represented as "5\333\333z~\272\232\277\275\026!L$\246n\317\365\353\226\262\353\2557\302\301\264\252m\341\213 n".

The invoice also includes some features. The "tlv-onion" feature allows for onion-encrypted payloads using the TLV format. The "payment-addr" feature is required and indicates that a payment address is included in the invoice. The "multi-path-payments" feature supports splitting the pay

In [17]:
invoice_to_pay = "lnsb10n1pjfzgscpp5gayl6r39ty0cnjsdf889aywn9qlvmr7msru48vjc29k5akde43hsdqqcqzzsxqyz5vqsp5mq4hutpexz6zp8kyfr6l9khxnx4gzkqsxrmgdxz0l9edk57am64q9qyyssq2j2hg7rzyfr4t9t0t2jqm7qq86atzz4w82l2zggvghq4zx5vcey3m93skdjwq58x63530ryke53tms920sj244mgftdthw54g22qwfgqq2kgfc"
prompt = "Pay this invoice, and return the preimage for ths invoice. Also can you tell me anything about the route the payment took, and how much in fees I paid: {invoice}".format(invoice=invoice_to_pay)
print(bitcoin_agent.run(prompt))



> Entering new  chain...

Invoking: `send_payment` with `{'invoice': 'lnsb10n1pjfzgscpp5gayl6r39ty0cnjsdf889aywn9qlvmr7msru48vjc29k5akde43hsdqqcqzzsxqyz5vqsp5mq4hutpexz6zp8kyfr6l9khxnx4gzkqsxrmgdxz0l9edk57am64q9qyyssq2j2hg7rzyfr4t9t0t2jqm7qq86atzz4w82l2zggvghq4zx5vcey3m93skdjwq58x63530ryke53tms920sj244mgftdthw54g22qwfgqq2kgfc'}`


payment_preimage: "/8\245\276@;\027\265\327\343\0259T\370\310$6\264.G\253*\025d\262z~\3777\373j}"
payment_route {
  total_time_lock: 3388
  total_amt: 1
  hops {
    chan_id: 3619592278704128
    chan_capacity: 1000000
    amt_to_forward: 1
    expiry: 3388
    amt_to_forward_msat: 1000
    pub_key: "038cf9360fa60a6de14ad7490950f90e06e828d4557a711aa54fabc6a5f2b4d81d"
    tlv_payload: true
    mpp_record {
      payment_addr: "\330+~,90\264 \236\304H\365\362\332\346\231\252\201X\0200\366\206\230O\371r\333S\335\336\252"
      total_amt_msat: 1000
    }
  }
  total_amt_msat: 1000
}
payment_hash: "GI\375\016%Y\037\211\312\rI\316^\221\323(>\315\217\333\200\371S\